In [183]:
from tensorflow.keras import datasets,utils
import numpy as np

In [184]:
(x_train,y_train),(x_test,y_test) = datasets.cifar10.load_data()

In [185]:
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

number_of_classes = 10
y_train = utils.to_categorical(y_train,number_of_classes)
y_test = utils.to_categorical(y_test,number_of_classes)

In [186]:
print(f"x_train:{x_train.shape} y_train{y_train.shape}")
print(f"x_test:{x_test.shape} y_test{y_test.shape}")

x_train:(50000, 32, 32, 3) y_train(50000, 10)
x_test:(10000, 32, 32, 3) y_test(10000, 10)


In [187]:
from tensorflow.keras import models,layers

In [188]:
class ModelCNN(models.Model):
    def __init__(self):
        super(ModelCNN,self).__init__()
        self.convolutional_layer1 = layers.Conv2D(10,kernel_size=(4,4),strides=2,padding="same")
        self.convolutional_layer2 = layers.Conv2D(20,kernel_size=(3,3),strides=2,padding="same")
        self.flatten_layer = layers.Flatten()
        self.output_layer = layers.Dense(units=10,activation="softmax")
    def call(self, inputs, training=None, mask=None):
        x = self.convolutional_layer1(inputs)
        x = self.convolutional_layer2(x)
        x = self.flatten_layer(x)
        x = self.output_layer(x)
        return x

In [189]:
model = ModelCNN()
model.build((None,32,32,3))
model.summary()

Model: "model_cnn_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_50 (Conv2D)          multiple                  490       
                                                                 
 conv2d_51 (Conv2D)          multiple                  1820      
                                                                 
 flatten_13 (Flatten)        multiple                  0         
                                                                 
 dense_17 (Dense)            multiple                  12810     
                                                                 
Total params: 15120 (59.06 KB)
Trainable params: 15120 (59.06 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [190]:
out = model.predict(x_train)

1563/1563 [==============================] - 1s 673us/step


In [191]:
print(out.shape)

(50000, 10)


In [192]:
input_layer = layers.Input((32,32,3))

x = layers.Conv2D(32,kernel_size=3,strides=1,padding="same")(input_layer)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(32,kernel_size=3,strides=2,padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(64,kernel_size=3,strides=1,padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(64,kernel_size=3,strides=2,padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)

x = layers.Flatten()(x)

x = layers.Dense(128)(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU()(x)
x = layers.Dropout(rate=0.5)(x)

output_layer = layers.Dense(10,activation="softmax")(x)

cnn_model = models.Model(input_layer,output_layer)

In [193]:
cnn_model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_52 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_30 (Ba  (None, 32, 32, 32)        128       
 tchNormalization)                                               
                                                                 
 leaky_re_lu_30 (LeakyReLU)  (None, 32, 32, 32)        0         
                                                                 
 conv2d_53 (Conv2D)          (None, 16, 16, 32)        9248      
                                                                 
 batch_normalization_31 (Ba  (None, 16, 16, 32)        128       
 tchNormalization)                                         

In [194]:
from tensorflow.keras import optimizers

In [195]:
opt = optimizers.legacy.Adam(learning_rate=0.0005)
cnn_model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [196]:
hist = cnn_model.fit(x_train,y_train,epochs=10)

Epoch 1/10
1563/1563 [==============================] - 29s 18ms/step - loss: 1.5346 - accuracy: 0.4629
Epoch 2/10
1563/1563 [==============================] - 29s 18ms/step - loss: 1.1264 - accuracy: 0.6022
Epoch 3/10
1563/1563 [==============================] - 32s 21ms/step - loss: 0.9926 - accuracy: 0.6528
Epoch 4/10
1563/1563 [==============================] - 30s 19ms/step - loss: 0.9123 - accuracy: 0.6829
Epoch 5/10
1563/1563 [==============================] - 32s 21ms/step - loss: 0.8472 - accuracy: 0.7052
Epoch 6/10
1563/1563 [==============================] - 30s 19ms/step - loss: 0.7957 - accuracy: 0.7225
Epoch 7/10
1563/1563 [==============================] - 29s 19ms/step - loss: 0.7589 - accuracy: 0.7351
Epoch 8/10
1563/1563 [==============================] - 31s 20ms/step - loss: 0.7149 - accuracy: 0.7498
Epoch 9/10
1563/1563 [==============================] - 35s 23ms/step - loss: 0.6827 - accuracy: 0.7607
Epoch 10/10
1563/1563 [==============================] - 36s 23m

In [197]:
cnn_model.evaluate(x_test,y_test)

313/313 [==============================] - 2s 7ms/step - loss: 0.8988 - accuracy: 0.6967


[0.8987581133842468, 0.6966999769210815]